In [7]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(".")))

import pandas as pd
from utils import GCSHelper
from tqdm import tqdm
import re
gcs_helper = GCSHelper("../keys/gcs_key.json")
gcs_helper_preprocessed = GCSHelper("../keys/gcs_key.json", bucket_name="maple_preprocessed_data")

In [95]:
drop_list = [
    "nickname",
    "codi-hat",
    "codi-hair",
    "codi-face",
    "codi-top",
    "codi-bottom",
    "codi-shoes",
    "codi-weapon",
    ]
inter_df = gcs_helper.read_df_from_gcs("csv/user_detail_total.csv").drop_duplicates(drop_list)
item_df = gcs_helper_preprocessed.read_df_from_gcs("item_KMST_1149_VER1.0.csv")
# https://maple.inven.co.kr/dataninfo/item/
# 여기서 찾아보면 id가 일치하는 것을 확인할 수 있음
# 아직 잘은 몰라도 id가 maplestory.io랑 inven이 일치하는거 보면
# 메이플에서 직접 지정되어 있는 id인듯?

In [96]:
# None인 애들에 공통점이 공용인것을 확인했으니 Beginner로 채워줌
item_df["requiredJobs"] = item_df["requiredJobs"].replace("None", "['Beginner']")

# 역시 0과 같은 것을 확인했으니 0으로 채워줌
item_df["requiredLevel"] = item_df["requiredLevel"].replace("None", 0)



In [97]:
# item.csv 전처리

# 먼저 이름이 그냥 None인 애들은 다 날려버리자. 29개의 이름이 None인 아이템이 있다.
item_drop_none = item_df[item_df["name"] != "None"]
item_drop_none = item_drop_none[item_drop_none["name"] != "-"]


# 특수문자, 공백을 없애자. 진짜로 이거로 구분되는 아이템이 있다면.. 유감...
# 이렇게 처리한 이름을 key로 사용하더라도, 보여줄 때는 본래의 이름으로 보여줄 수 있도록 원래 name도 유지한다. 
# 40524 개의 아이템에서 37558개의 아이템으로 줄어들었다. 
# pattern = r'[!@#$%^&*()_+-=`~,.<>/?\s]'
pattern = r'[!@#$%^&*()_+-=`~,.<>/?{}\s\[\]]'
item_drop_none["name_processed"] = item_drop_none["name"].apply(lambda x: re.sub(pattern, "", x.lower()))
item_drop_none["name_processed"].nunique()

# 그 다음 이름이 동일한 아이템 리스트를 뽑아보자.
# 겹치는 이름은 6617개.
item_same_name = item_drop_none["name_processed"].value_counts()[item_drop_none["name_processed"].value_counts() != 1].index
item_same_name.nunique()

# 해당 아이템 그냥 한개만 남기고 다 날리자~
# 날리고 나면 37556개의 row가 남는다. 
# 이제 이 값은 이름의 nunique와 동일하다. 
item_drop_duplicate = item_drop_none.drop_duplicates(["name_processed"]).reset_index(drop=True)

In [103]:
item_drop_duplicate["subCategory"].unique()

array(['Hat', 'Top', 'Face', 'Hair', 'Overall', 'Bottom', 'Shoes',
       'Shining Rod', 'Bladecaster', 'Breath Shooter', 'Soul Shooter',
       'Desperado', 'Whip Blade', 'Psy-limiter', 'Chain', 'Gauntlet',
       'Ritual Fan', 'One-Handed Sword', 'One-Handed Axe',
       'One-Handed Blunt Weapon', 'Dagger', 'Katara', 'Cane', 'Wand',
       'Staff', 'Two-Handed Sword', 'Two-Handed Axe', 'Two-Handed Blunt',
       'Spear', 'Pole Arm', 'Bow', 'Crossbow', 'Claw', 'Knuckle', 'Gun',
       'Dual Bowgun', 'Hand Cannon', 'Lapis', 'Lazuli', 'Arm Cannon',
       'Ancient Bow', 'Cash'], dtype=object)

In [139]:
# inter에 있는 아이템 모아보기
# 마찬가지로 특수 문자 및 공백을 제거했다. 
# 소문자로도 변경하자.
# 6796개의 unique한 아이템들이 나왔다. 

item_list_from_maple_gg = [
    inter_df["codi-hat"],
    inter_df["codi-hair"],
    inter_df["codi-face"],
    inter_df["codi-top"],
    inter_df["codi-bottom"],
    inter_df["codi-shoes"],
    inter_df["codi-weapon"],
]
pattern = r'[!@#$%^&*()_+-=`~,.<>/?{}\s\[\]]'
item_list_from_maple_gg = pd.concat(item_list_from_maple_gg).reset_index(drop=True).apply(lambda x: re.sub(pattern, "", x.lower()))
item_list_from_maple_gg = item_list_from_maple_gg[item_list_from_maple_gg != ""].unique()

In [141]:
# 이제 maplestory.io와 maple.gg의 데이터를 묶어보자. 
# 6796 개의 아이템 중, maplestory.io에 없는 놈들을 찾아보자. 
# 760개의 아이템은 이름이 정확히 일치하지 않고, 나머지 6036개는 이름이 정확히 일치한다. 

item_set_from_maplestory_io = set(item_drop_duplicate["name_processed"].tolist())
not_in_maplestory_io = set()
can_use_name_directly_to_key = set()

for item_name_from_maple_gg in item_list_from_maple_gg:
    if item_name_from_maple_gg not in item_set_from_maplestory_io:
        not_in_maplestory_io.add(item_name_from_maple_gg)
    else:
        can_use_name_directly_to_key.add(item_name_from_maple_gg)

In [143]:
# 이름이 정확히 일치하지 않는 것은 ~~색 헤어 이런 것 때문인 것으로 추정.
# ~~색 이런 것을 고려해 완전히 일치하는 것이 아니라 포함 관계에 있는지 확인해보자.

# 760 개의 아이템 중, 744개의 아이템은 이름이 포함되어 있음을 확인함.
# 이 744개의 아이템을 포함하는 maplestory.io 출신 아이템 이름은 6073개


# 760 개의 아이템 중, 포함 관계로 key를 만들 수 있는 maple.gg의 아이템들
in_maplestory_io_use_contain = set()


# maplestory.io의 아이템 중 maple.gg의 아이템 이름과 동일하지는 않으나
# maple.gg의 아이템 이름을 포함하고 있는 아이템 이름
# ex. 칼날 헤어 in maple.gg, 검은색 칼날 헤어 in maplestory.io
can_use_name_to_key_with_small_fix = set()

# 798 개 아이템 하나씩 확인
for item_name_from_maple_gg in tqdm(not_in_maplestory_io):
    # maplestory.io의 아이템들이랑 하나씩 비교
    for item_name_from_maplestory_io in item_set_from_maplestory_io:

        if item_name_from_maple_gg in item_name_from_maplestory_io:
            in_maplestory_io_use_contain.add(item_name_from_maple_gg)
            can_use_name_to_key_with_small_fix.add(item_name_from_maplestory_io)

100%|██████████| 761/761 [00:01<00:00, 523.59it/s]


In [145]:
# 상남자 특) ~~ 색 그냥 지워버림
# 일단 무턱대고 다 지우기 보다는 can_use_name_to_key_with_small_fix 여기 속한 애들만 좀 지워보자. 
# 6073개의 아이템들에서 ~~색 이라는 부분을 싹 다 지워서 item_drop_color를 만들어봤다. 
item_drop_color = item_drop_duplicate.copy()
pattern = r'^.{2}색'
for item_name_from_maplestory_io in tqdm(can_use_name_to_key_with_small_fix):
    item_drop_color.loc[item_drop_color["name_processed"] == item_name_from_maplestory_io, "name_processed"] = re.sub(pattern, "", item_name_from_maplestory_io)

item_drop_color = item_drop_color.drop_duplicates(["name_processed"]).reset_index(drop=True)


100%|██████████| 5878/5878 [00:09<00:00, 599.47it/s]


In [146]:
# 이제 새롭게 만들어진 item_drop_color를 이용해서 위의 과정을 그대로 반복해보자.
# 28개가 아직도 짝이 안맞고, 6768개는 이제 이름을 곧바로 key로 사용할 수 있다. 

item_set_from_maplestory_io = set(item_drop_color["name_processed"].tolist())
not_in_maplestory_io = set()
can_use_name_directly_to_key = set()

for item_name_from_maple_gg in item_list_from_maple_gg:
    if item_name_from_maple_gg not in item_set_from_maplestory_io:
        not_in_maplestory_io.add(item_name_from_maple_gg)
    else:
        can_use_name_directly_to_key.add(item_name_from_maple_gg)


In [147]:
not_in_maplestory_io.__len__(), can_use_name_directly_to_key.__len__(), item_set_from_maplestory_io.__len__()

(21, 6774, 12350)

In [151]:
# 28개가 하나라도 포함된 row가 몇개나 있는지 확인해보자. 
# 130만개중, 7245개의 row에 해당 아이템들이 포함된다. 
# 약 0.5%의 비중이므로 그냥 무시하자. 
# 이름, 장비가 모두 같은 경우를 drop했을때는 약 0.58%의 비중을 갖는다. 무시해도 된다!

row_not_matched = []
pattern = r'[!@#$%^&*()_+-=`~,.<>/?{}\s\[\]]'


for i, row in tqdm(inter_df.iterrows()):
    codi_list = [
        re.sub(pattern, "", row["codi-hat"].lower()),
        re.sub(pattern, "", row["codi-hair"].lower()),
        re.sub(pattern, "", row["codi-face"].lower()),
        re.sub(pattern, "", row["codi-top"].lower()),
        re.sub(pattern, "", row["codi-bottom"].lower()),
        re.sub(pattern, "", row["codi-shoes"].lower()),
        re.sub(pattern, "", row["codi-weapon"].lower())
    ]
    for codi_item in codi_list:
        if codi_item == "":
            continue

        if codi_item not in item_set_from_maplestory_io:
            row_not_matched.append(row)
            break

1015758it [01:28, 11531.36it/s]


In [153]:
len(row_not_matched)

1201

In [154]:
item_drop_color["requiredGender"] = item_drop_color["requiredGender"].apply(lambda x: min(x, 2))

In [155]:
gcs_helper_preprocessed.upload_df_to_gcs("item_KMST_1149_VER1.1.csv", item_drop_color)

### 흐름 정리

 - 현재 가지고 있는 130만개의 데이터를 이용해서 item.csv에 name_processed column을 새로 만든다. 
 - 해당 컬럼은 다음과 같은 프로세싱 과정을 거친 결과다.
   - name column에서 특수문자 및 공백을 모두 제거한다. 
   - 영어는 모두 소문자로 변환한다.
   - 130만개의 데이터에 대해서 maple.gg와 maplestory.io의 아이템 이름이 정확히 일치하지 않는 경우의 maple.gg의 아이템 리스트를 골라낸다. 
   - 해당하는 아이템 리스트는 "~~색 무슨 헤어"에서 "~색" 때문인 경우가 매우 많다.
   - 이 "~색" 을 제거하면 이름이 정확히 일치하는 경우가 많다. 
   - 따라서 이런 처리를 해줬을 때 정확히 일치하게 되는 maplestory.io의 아이템 이름들에 대해서, "~색"을 제거한다. 
 - 이 과정을 통해 얻은 name_processed column은 maple.gg의 코디 정보에서 특수문자, 공백을 제거한 이름을 99.5%까지 커버 가능하다. 
 - 따라서, 0.5% 가량은 무시하고, 이렇게 얻은 name_processed column을 key로 사용한다. 
   - 일치하지 않은 경우, row는 버린다. 
   - 추후, data가 더 모이면 물론 더 정확한 name_processed column을 만들 수 있다. 하지만, 무시한다. 